In [12]:
def converter(input_files, output_file, station_filter):
    event_counter = 0  # Initialize an event counter

    # Read station names from the text file into a list
    with open(station_filter, 'r') as station_file:
        allowed_stations = [line.strip() for line in station_file]

    # Open the output file for writing
    with open(output_file, 'w') as outfile:
        # Loop through each input file
        for fileinput in input_files:
            with open(fileinput, 'r') as infile:
                # Read all lines from the input file and split each line into a list of words
                lines = infile.readlines()
                for i in range(len(lines)):
                    lines[i] = lines[i].split()

                i = 0
                while i < len(lines):
                    if len(lines[i]) > 0 and lines[i][0] == 'EventID:':
                        # Extract information from the seismic event record
                        year = lines[i + 2][0].split('-')[0]
                        month = lines[i + 2][0].split('-')[1].zfill(2)
                        date = lines[i + 2][0].split('-')[2].zfill(2)
                        hour = lines[i + 2][1].split(':')[0].zfill(2)
                        minute = lines[i + 2][1].split(':')[1].zfill(2)
                        second = (('%.1f') % float(lines[i + 2][1].split(':')[2])).zfill(4)
                        latitude = (('%.2f') % float(lines[i + 2][2])).zfill(6)
                        longitude = (('%.2f') % float(lines[i + 2][3])).zfill(6)
                        
                        
                        if -12 <= float(latitude) <= -4 and 104 <= float(longitude) < 107.0:  # Apply a filter based on latitude and longitude
                            depth = ('%.1f') % float(lines[i + 2][4])
                            mag = ('%.1f') % float(lines[i + 2][5])
                            unknown = '0.0'
                            rms = ('%.3f') % float(lines[i + 2][10])
                            time0 = float(second) + float(minute) * 60 + float(hour) * 60 * 60 + float(date) * 60 * 60 * 24
                            event_counter += 1
                            # Write the extracted and filtered information to the output file
                            outfile.write('#' + year.rjust(5) + month.rjust(3) + date.rjust(3) + hour.rjust(3) +
                                          minute.rjust(3) + second.rjust(5) + latitude.rjust(8) + longitude.rjust(10) +
                                          depth.rjust(9) + mag.rjust(5) + unknown.rjust(6) + unknown.rjust(6) +
                                          rms.rjust(8) + str(event_counter).rjust(10) + '\n')

                    if len(lines[i]) > 0 and lines[i][0] == 'Net' and -12 <= float(latitude) <= -4 and 104 <= float(longitude) < 107.0: # apply the filter in this line too
                        try:
                            j = 0
                            while j < 1:
                                i = i + 1
                                try:
                                    station = lines[i][1]
                                    if station in allowed_stations:
                                        phase = lines[i][2]
                                        date = lines[i][3].split('-')[2]
                                        hour = lines[i][4].split(':')[0]
                                        minute = lines[i][4].split(':')[1]
                                        second = lines[i][4].split(':')[2]
                                        time1 = float(second) + float(minute) * 60 + float(hour) * 60 * 60 + float(date) * 60 * 60 * 24
                                        deltatime = ('%.2f') % (time1 - time0)
                                        unknown = '1.000'
                                        
                                        # Write station-specific information to the output file
                                        outfile.write(station.rjust(5) + deltatime.rjust(12) + unknown.rjust(8) +
                                                      phase.rjust(4) + '\n')
                                except:
                                    break
                        except:
                            break
                    i = i + 1


In [7]:
# Example usage:
input_files = ['2012.txt','2013.txt','2014.txt','2015.txt','2016.txt','2017.txt']  # Add your input file paths here
output_file = '2012-2017.pha'
station_filter = 'filter_station.txt'
converter(input_files, output_file, station_filter)

In [5]:
def edit_event_phase(phase, header, output_file):
    with open(phase, 'r') as f1, open(header, 'r') as f2, open(output_file, 'w') as output:
        data = f1.readlines()
        for i, line in enumerate(data):
            if '#' in line and len(line.split()) > 5:
                output.write(next(f2).strip() + '\n')
            else:
                output.write(f"{line.split()[0]}   {line.split()[1]}    {line.split()[2]}   {line.split()[3]}\n")



In [8]:
# Usage example:
phase = '2012-2017.pha'
header = 'new_header.txt'
output_file ='absolute.dat'
edit_event_phase(phase, header, output_file)

In [ ]:
with open('2012-2017_filtered.pha', 'r') as f:
    lines = f.readlines()

new_lines = []
for i, line in enumerate(lines):
    if line.startswith('#'):
        new_lines.append(line)
        continue
    station = line.split()[0]
    event_lines = [line]
    for next_line in lines[i+1:]:
        if next_line.startswith('#'):
            break
        next_station = next_line.split()[0]
        if next_station == station:
            event_lines.append(next_line)
    has_p = False
    has_s = False
    if len(event_lines) == 2:
        for event_line in event_lines:
            if event_line.split()[3] == 'P':
                has_p = True
            elif event_line.split()[3] == 'S':
                has_s = True
        if has_p and has_s:
            new_lines.extend(event_lines)
    elif len(event_lines) == 4:
        new_lines.extend(event_lines)

for elem in new_lines:
    print(elem)

In [ ]:
print("hello git")